<a href="https://colab.research.google.com/github/TheNewLearn/bd_hw3/blob/main/bd_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [165]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as f

spark = SparkSession.builder.master('local').appName('hw3').config("spark.jars.packages","com.databricks:spark-xml_2.12:0.13.0").getOrCreate()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libnss-mdns fonts-dejavu-extra fonts-ipafont-gothic fonts-ipafont-mincho
  fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jre-headless
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 28.2 MB of archives.
After this operation, 104 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 openjdk-8-jre-headless amd64 8u292-b10-0ubuntu1~18.04 [28.2 MB]
Fetched 28.2 MB in 2s (16.2 MB/s)
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../openjdk-8-jre-headless_8u292-b10-0ubuntu1~18.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u292-b10-0ubuntu1~18.04) ...
Setting up openjdk-8-jre-headless:amd64 (8u292-b10-0ubunt

In [166]:
from google.colab import drive
drive.mount("/content/drive")
path = "/content/drive/My Drive/bd_hw3/reut2-000.sgm"

Mounted at /content/drive


In [167]:
import pyspark.sql.functions as f

def k_shingling(text,k):
  t = text.split()
  ls = [' '.join(t[i:i+k])
                     for i in range(len(t) - k + 1)]
  return ls

doc = []

for i in range(0,22):
  doc.append([])
  if i < 10:
    dataset2 = spark.read.format('xml')\
            .option('rootTag',"")\
            .option('rowTag','BODY')\
            .load("/content/drive/My Drive/bd_hw3/reut2-00"+str(i)+".sgm")
  elif i >= 10:
    dataset2 = spark.read.format('xml')\
            .option('rootTag',"")\
            .option('rowTag','BODY')\
            .load("/content/drive/My Drive/bd_hw3/reut2-0"+str(i)+".sgm")
  newdataset2 = dataset2.select("_corrupt_record", f.regexp_replace(f.col("_corrupt_record"), "[\$#,<BODY></BODY>;'\''/']", "").alias("replaced"))
  content2 = newdataset2.rdd.map(lambda x:x[1].replace("\n","")).map(lambda x:x.replace("&3","")).map(lambda x:x.replace("  ","")).map(lambda x:x.replace("&lt",""))
  doc[i] += content2.collect()



def compare(x,ls):
  for i in ls:
    if x in i:
      return "1"
    else:
      return "0"

allks = []

for i in range(0,22):
  if i < 10:
    dataset = spark.read.format('xml')\
            .option('rootTag',"")\
            .option('rowTag','BODY')\
            .load("/content/drive/My Drive/bd_hw3/reut2-00"+str(i)+".sgm")
  elif i >= 10:
    dataset = spark.read.format('xml')\
            .option('rootTag',"")\
            .option('rowTag','BODY')\
            .load("/content/drive/My Drive/bd_hw3/reut2-0"+str(i)+".sgm")

  newdataset = dataset.select("_corrupt_record", f.regexp_replace(f.col("_corrupt_record"), "[\$#,<BODY></BODY>;'\''/']", "").alias("replaced"))
  content = newdataset.rdd.map(lambda x:x[1].replace("\n","")).map(lambda x:x.replace("&3","")).map(lambda x:x.replace("  ","")).map(lambda x:x.replace("&lt",""))
  content = content.flatMap(lambda x: k_shingling(x,3))
  allks += content.collect()

In [239]:
#task1 matrix
col = []
col.append("    ")
data2 = []
for column in range(len(doc)):
  col.append("Doc"+str(column))

for row in range(0,1000):
  data2.append([])
  data2[row].append(str(allks[row]))
  for column in range(len(doc)):
    match = 0
    for index in doc[column]:
      if allks[row] in index:
        match = 1
    data2[row].append(str(match))


rdd = spark.sparkContext.parallelize(data2)

df = spark.createDataFrame(rdd).toDF(col[0],col[1],col[2],col[3],col[4],col[5],col[6],col[7],col[8],col[9],col[10],col[11],col[12],col[13],col[14],col[15],col[16],col[17],col[18],col[19],col[20],col[21],col[22])

df.show(100)


+--------------------+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|                    |Doc0|Doc1|Doc2|Doc3|Doc4|Doc5|Doc6|Doc7|Doc8|Doc9|Doc10|Doc11|Doc12|Doc13|Doc14|Doc15|Doc16|Doc17|Doc18|Doc19|Doc20|Doc21|
+--------------------+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|Showers continued...|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|
|continued through...|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|
| throughout the week|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    0|
|      the week inthe|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|    0|    0|    0|    0|    0|    0|    0|    0|    0|    

In [169]:
import random

maxShingleID = 2**32-1

def pickRandomCoeffs(k):
  # Create a list of 'k' random values.
  randList = []
  
  while k > 0:
    # Get a random shingle ID.
    randIndex = random.randint(0, maxShingleID) 
  
    # Ensure that each random number is unique.
    while randIndex in randList:
      randIndex = random.randint(0, maxShingleID) 
    
    # Add the random number to the list.
    randList.append(randIndex)
    k = k - 1
    
  return randList



print(pickRandomCoeffs(10))
print(data2)


[1669360650, 3697042388, 1045979734, 227353940, 1219840041, 3587886447, 3453852794, 270828717, 327011366, 3303885087]
[['Showers continued throughout', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['continued throughout the', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['throughout the week', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['the week inthe', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['week inthe ahia', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['inthe ahia cocoa', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['ahia cocoa zone', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0',

In [319]:
import random
maxnum = 2**32-1

def minhash(k):
#k hash function
  hn = []
  for i in range(0,len(data2)):
    hn.append([])
    for j in range(k):
      a = random.randint(0, maxnum)
      b  = random.randint(0, maxnum)
      rowindex =  (a*i+b)%maxnum
      hn[i].append(rowindex)
  ms = []
  for k in range(len(hn[0])):
    ms.append([])
    for i in range(0, len(data2[0])):
      min = maxnum
      for row in range(len(data2)):
        if data2[row][i] == "1":
          if min > hn[row][k]:
            min = hn[row][k]
        else:
          continue
      ms[k].append(min)

    
  return ms


a = minhash(2)
c = minhash(1)

for i in range(len(a)):
  for j in a[i]:
    print(j,end=" ")
  print(" ")

print(len(c))




4294967295 14664604 102817572 75908391 75908391 68268447 75908391 23175880 75908391 75908391 68268447 68268447 75908391 75908391 75908391 102817572 23175880 23175880 75908391 102817572 75908391 75908391 102817572  
4294967295 931666 4646635 4646635 4646635 5776147 20177522 4646635 4646635 4646635 11675924 4646635 4646635 4646635 4646635 4646635 4646635 4646635 4646635 4646635 4646635 39009024 4646635  
1


In [121]:


for i in range(0,len(data2)):
  minindex = maxnum
  for j in range(0,len(data2[i])):
    if data2[i][j] == "1":
      for k in range(0,len(a[i])):
        if a[i][k] < minindex:
          minindex = a[i][k]
      
      print(minindex,end=" ")
  print(" ")

1  
2  
3  
4  
5  
6  
7  
8  
9  
10  
11  
12  
13  
14  
15  
16  
17 17 17 17 17 17 17 17 17 17 17 17  
18 18 18 18 18 18 18 18 18 18 18 18  
19  
0  


In [173]:



d = [[1,0,0,1],
     [0,0,1,0],
     [0,1,0,1],
     [1,0,1,1],
     [0,0,1,0]]


f = [[1,1],
     [2,4],
     [3,2],
     [4,0],
     [0,3]]

sc = []



min = 20


for k in range(len(f[0])):
  sc.append([])
  for i in range(0, len(d[0])):
    min = 20
    for row in range(len(d)):
      if d[row][i] == 1:
        if min > f[row][k]:
          min = f[row][k]
      else:
        continue
    sc[k].append(min)
  


print(sc)     

    


[[1, 3, 0, 1], [0, 2, 0, 0]]


In [320]:

def lsh(b,r,sign):
  band = []
  for i in range(0,b):
    band.append([])
    for k in range(0, len(sign[0])):
      for row in range(0,len(sign)):
        band[i].append(sign[row][k])
  return band

print(a[0])
#a = lsh(2,2,c)
b = lsh(2,2,c)

print(a)

[4294967295, 14664604, 102817572, 75908391, 75908391, 68268447, 75908391, 23175880, 75908391, 75908391, 68268447, 68268447, 75908391, 75908391, 75908391, 102817572, 23175880, 23175880, 75908391, 102817572, 75908391, 75908391, 102817572]
[[4294967295, 14664604, 102817572, 75908391, 75908391, 68268447, 75908391, 23175880, 75908391, 75908391, 68268447, 68268447, 75908391, 75908391, 75908391, 102817572, 23175880, 23175880, 75908391, 102817572, 75908391, 75908391, 102817572], [4294967295, 931666, 4646635, 4646635, 4646635, 5776147, 20177522, 4646635, 4646635, 4646635, 11675924, 4646635, 4646635, 4646635, 4646635, 4646635, 4646635, 4646635, 4646635, 4646635, 4646635, 39009024, 4646635]]
